In [257]:
# Plonk a poly IOP for a general circuit.
# Step 1: Compile a circuit to a computationa trace (gate fan-in = 2)

# Starting from the (x_1+x_2)(x_2+w_1)
# x_1 and x_2 are the inputs, w_1 is a witness
# We can compile it to the following trace:

#.             L     R     O
# | inputs |   5  |  6  |  1 | (input values)
# | gate 0 |   5  |  6  | 11 | (5+6 = 11)
# | gate 1 |   6  |  1  | 7  | (6+1 = 7)
# | gate 2 |  11  |  7  | 77 | (11*7 = 77)

# |C| := total # of gates in C
# |I| := |Ix| + |Iw| = total # of inputs (public + witness)
# d:= 3|C|+|I| (3*4 = 12)
# Omega := {1,w,w^2,...,w^(d-1)} (a size-d multiplicative subgroup of the field)

In [258]:
# Setting up out prime field, using small example with p=101 for ease of use 
import galois
GF = galois.GF(193)
# Setting up our multiplicative subgroup of size d=12
d = 12
w = GF.primitive_root_of_unity(d) # find the 12-th root of unity
print("The 12-th root of unity is:", w)
Omega = [w**i for i in range(d)]
omega_field = GF(Omega)
# output: [1, 49, 85, 112, 84, 63, 192, 144, 108, 81, 109, 130]
print(omega_field)
# this works since multiplying 49 continuosly cycles through the list
# 1*49=49
# 49*49=85
# 85*49=112
# ...
# 130*49=1

The 12-th root of unity is: 49
[  1  49  85 112  84  63 192 144 108  81 109 130]


In [259]:
# Prover interpolates T(X) such that 
# inputs: T(w^-1) = 5,   T(w^-2) = 6,   T(w^-3) = 1     inputs
# gate 0: T(w^0)  = 5,   T(w^1)  = 6,   T(w^2)  = 11    gate 0
# gate 1: T(w^3)  = 6,   T(w^4)  = 1,   T(w^5)  = 7     gate 1
# gate 2: T(w^6)  = 11,  T(w^7)  = 7,   T(w^8)  = 77    gate 2
Y = GF([5, 6, 11, 6, 1, 7, 11, 7, 77, 1, 6, 5])
T_X = galois.lagrange_poly(omega_field, Y)
print("T(X):", T_X)
# evaluate in w^3 as an exmample
print("T(w^3):", T_X(w**-1))  # should be 5

T(X): 113x^11 + 141x^10 + 112x^9 + 93x^8 + 64x^7 + 87x^6 + 39x^5 + 126x^4 + 58x^3 + 112x^2 + 190x + 28
T(w^3): 5


In [260]:
# Prove that T encodes the correct inputs for j = 1, ... , |Ix| = 2
Omega_inp = omega_field[-2:]  # last two elements of Omega
print("Input positions in Omega:", Omega_inp)
# Encode to the input values
Y_inp = GF([6, 5])  # input values
# Interpolate the input polynomial
V_inp_X = galois.lagrange_poly(Omega_inp, Y_inp)
print("V_inp(X):", V_inp_X)
# ZeroTest on the input domain
assert(T_X(w**-1) - V_inp_X(w**-1) == 0)
assert(T_X(w**-2) - V_inp_X(w**-2) == 0)

Input positions in Omega: [109 130]
V_inp(X): 147x + 2


In [261]:
# Now we must prove that every gate is evaluated correctly
# Encoding the selector polynomial S(X) where for l=0,...,|C|-1:
# S(w^{3l}) = 1 (for addition gate)
# S(w^{3l}) = 0 (for a multiplication gate)
# Adding new selector values 
#.             L     R     O   S
# | inputs |   5  |  6  |  1 |   |
# | gate 0 |   5  |  6  | 11 | 1 |
# | gate 1 |   6  |  1  | 7  | 1 |
# | gate 2 |  11  |  7  | 77 | 0 |

# Difining omega_gates as every third element of omega_field until 3C-1
omega_gates = omega_field[0:9:3]  # positions of the gates
print("Omega gates:", omega_gates)
S_X = galois.lagrange_poly(omega_gates, GF([1, 1, 0]))
print("S(X):", S_X) 
# zerotest on all gates 
for i in omega_gates:
    assert S_X(i)*(T_X(i* w**0) + T_X(i* w**1)) + (GF(1)-S_X(i))*T_X(i* w**0) * T_X(i* w**1) - T_X(i* w**2) == 0


Omega gates: [  1 112 192]
S(X): 76x^2 + 97x + 21


In [262]:
# Prove that the wiring is correct
# example: x1=5, x2=6 , w1=1

#       w^-1, w^-2, w^-3   : 5, 6, 1
#                               |
# 0:    w^0 , w^1 , w^2    : 5, 6, 11
#                              /
# 1:    w^3 , w^4 , w^5    : 6, 1, 7
# 2:    w^6 , w^7 , w^8    : 11, 7, 77
# as we can see the wiring is as follows:
# T(ω^-2) = T(ω^1) = T(ω^3)
# T(ω^-1) = T(ω^0)
# T(ω^2) = T(ω^6)
# T(ω^-3) = T(ω^4)

# W(w^-2, w^1, w^3) = (w^1,w^3,w^-2)
# W(w^-1, w^0) = (w^0, w^-1)
# W(w^2, w^6) = (w^6, w^2)
# W(w^-3, w^4) = (w^4, w^-3)

# [  1  49  85 112  84  63 192 144 108  81 109 130]
# [  1  112  85 109  84  63 192 144 108  81 49 130], W(w^-2, w^1, w^3) = (w^1,w^3,w^-2)
# [  1  112  192 109  84  63 85 144 108  81 49 130], W(w^2, w^6) = (w^6, w^2)

# [  1  112  85 109  81  63 192 144 108  84 49 130], W(w^-3, w^4) = (w^4, w^-3)
W = GF([1 , 112 , 85, 109,  81,  63, 192, 144, 108,  84, 49, 130])
W_poly = galois.lagrange_poly(omega_field, W)
print("W(X):", W_poly)
 
 

 # Step A: Collect the "roots" (the values of the functions)
# f(a) for all a in Omega
roots_f = [T_X(y) for y in omega_field]

# g(a) = T(W(a)) for all a in Omega
roots_g = [T_X(W_poly(y)) for y in omega_field]

# Step B: Build the polynomials from these roots using galois.Poly.Roots
# This expands the product (X - r1)(X - r2)...
f_hat = galois.Poly.Roots(roots_f, field=GF)
g_hat = galois.Poly.Roots(roots_g, field=GF)

# Permutation check
Q, R = divmod(f_hat , g_hat)
assert Q == 1
assert R == 0



W(X): 166x^11 + 24x^10 + 118x^9 + 64x^7 + 86x^6 + 53x^5 + 181x^3 + 83x^2 + 191x


In [ ]:
# output test (evaluate T at w^{3|C|-1} and the value should be 77 since it's the output of the last gate
# more commonly the output would be 0 since it can be subtracted)
T_X(w**8)

GF(77, order=193)